Markdown

In [1]:
#Import relevant libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import requests
import os
from collections import Counter
import string

os.path.abspath(os.getcwd())

'/Users/michelleyuu/Desktop/data230/project'

Some code useage references from existing projects:
https://www.kaggle.com/morrisb/eda-with-interactive-maps-for-us-gun-violence

In [2]:
data = pd.read_csv("gun-violence-data_01-2013_03-2018.csv")

In [3]:
data

,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,...,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district
0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||3::Male||4::Female,0::Julian Sims,NaN,0::Arrested||1::Injured||2::Injured||3::Injure...,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://pittsburgh.cbslocal.com/2013/01/01/4-pe...,NaN,NaN
1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male,0::Bernard Gillis,NaN,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://losangeles.cbslocal.com/2013/01/01/man-...,62.0,35.0
2,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,http://www.gunviolencearchive.org/incident/478855,http://chronicle.northcoastnow.com/2013/02/14/...,False,...,0::25||1::31||2::33||3::34||4::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||2::Male||3::Male||4::Male,0::Damien Bell||1::Desmen Noble||2::Herman Sea...,NaN,"0::Injured, Unharmed, Arrested||1::Unharmed, A...",0::Subject-Suspect||1::Subject-Suspect||2::Vic...,http://www.morningjournal.com/general-news/201...,56.0,13.0
3,478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,http://www.gunviolencearchive.org/incident/478925,http://www.dailydemocrat.com/20130106/aurora-s...,False,...,0::29||1::33||2::56||3::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Female||1::Male||2::Male||3::Male,0::Stacie Philbrook||1::Christopher Ratliffe||...,NaN,0::Killed||1::Killed||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://denver.cbslocal.com/2013/01/06/officer-...,40.0,28.0
4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,http://www.gunviolencearchive.org/incident/478959,http://www.journalnow.com/news/local/article_d...,False,...,0::18||1::46||2::14||3::47,0::Adult 18+||1::Adult 18+||2::Teen 12-17||3::...,0::Female||1::Male||2::Male||3::Female,0::Danielle Imani Jameison||1::Maurice Eugene ...,3::Family,0::Injured||1::Injured||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://myfox8.com/2013/01/08/update-mother-sho...,62.0,27.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239672,1083142,2018-03-31,Louisiana,Rayne,North Riceland Road and Highway 90,0,0,http://www.gunviolencearchive.org/incident/108...,http://www.klfy.com/news/local/rayne-woman-cha...,False,...,0::25,0::Adult 18+,0::Female,0::Jhkeya Tezeno,NaN,"0::Unharmed, Arrested",0::Subject-Suspect,http://www.klfy.com/news/local/rayne-woman-cha...,NaN,NaN
239673,1083139,2018-03-31,Louisiana,Natchitoches,247 Keyser Ave,1,0,http://www.gunviolencearchive.org/incident/108...,http://www.ksla.com/story/37854648/man-wanted-...,False,...,1::21,0::Adult 18+||1::Adult 18+,0::Male||1::Male,0::Jamal Haskett||1::Jaquarious Tyjuan Ardison,NaN,"0::Killed||1::Unharmed, Arrested",0::Victim||1::Subject-Suspect,http://www.ksla.com/story/37854648/man-wanted-...,23.0,31.0
239674,1083151,2018-03-31,Louisiana,Gretna,1300 block of Cook Street,0,1,http://www.gunviolencearchive.org/incident/108...,http://www.nola.com/crime/index.ssf/2018/04/sh...,False,...,0::21,0::Adult 18+,0::Male,NaN,NaN,0::Injured,0::Victim,http://www.nola.com/crime/index.ssf/2018/04/sh...,85.0,7.0
239675,1082514,2018-03-31,Texas,Houston,12630 Ashford Point Dr,1,0,http://www.gunviolencearchive.org/incident/108...,https://www.ch

In [4]:
data.dtypes

incident_id                      int64
date                            object
state                           object
city_or_county                  object
address                         object
n_killed                         int64
n_injured                        int64
incident_url                    object
source_url                      object
incident_url_fields_missing       bool
congressional_district         float64
gun_stolen                      object
gun_type                        object
incident_characteristics        object
latitude                       float64
location_description            object
longitude                      float64
n_guns_involved                float64
notes                           object
participant_age                 object
participant_age_group           object
participant_gender              object
participant_name                object
participant_relationship        object
participant_status              object
participant_type         

In [4]:
#removed source url columns from data
#removed district columns
#removed incident id (all entry is considered a unique case)
#removed participant name (Unrequired information for visualization)
data.drop(columns=['incident_url', 'source_url', 'sources', 'incident_url_fields_missing', 
    'incident_id', 'congressional_district', 'state_house_district','state_senate_district',
    'participant_name'], inplace = True)

In [11]:
data

,date,state,city_or_county,address,n_killed,n_injured,gun_stolen,gun_type,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_relationship,participant_status,participant_type
0,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,NaN,NaN,Shot - Wounded/Injured||Mass Shooting (4+ vict...,40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||3::Male||4::Female,NaN,0::Arrested||1::Injured||2::Injured||3::Injure...,0::Victim||1::Victim||2::Victim||3::Victim||4:...
1,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,NaN,NaN,"Shot - Wounded/Injured||Shot - Dead (murder, a...",33.9090,NaN,-118.3330,NaN,Four Shot; One Killed; Unidentified shooter in...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male,NaN,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...
2,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,0::Unknown||1::Unknown,0::Unknown||1::Unknown,"Shot - Wounded/Injured||Shot - Dead (murder, a...",41.4455,Cotton Club,-82.1377,2.0,NaN,0::25||1::31||2::33||3::34||4::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||2::Male||3::Male||4::Male,NaN,"0::Injured, Unharmed, Arrested||1::Unharmed, A...",0::Subject-Suspect||1::Subject-Suspect||2::Vic...
3,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,NaN,NaN,"Shot - Dead (murder, accidental, suicide)||Off...",39.6518,NaN,-104.8020,NaN,NaN,0::29||1::33||2::56||3::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Female||1::Male||2::Male||3::Male,NaN,0::Killed||1::Killed||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...
4,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,0::Unknown||1::Unknown,0::Handgun||1::Handgun,"Shot - Wounded/Injured||Shot - Dead (murder, a...",36.1140,NaN,-79.9569,2.0,Two firearms recovered. (Attempted) murder sui...,0::18||1::46||2::14||3::47,0::Adult 18+||1::Adult 18+||2::Teen 12-17||3::...,0::Female||1::Male||2::Male||3::Female,3::Family,0::Injured||1::Injured||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239672,2018-03-31,Louisiana,Rayne,North Riceland Road and Highway 90,0,0,0::Unknown,0::Unknown,Shots Fired - No Injuries,NaN,NaN,NaN,1.0,NaN,0::25,0::Adult 18+,0::Female,NaN,"0::Unharmed, Arrested",0::Subject-Suspect
239673,2018-03-31,Louisiana,Natchitoches,247 Keyser Ave,1,0,0::Unknown,0::Unknown,"Shot - Dead (murder, accidental, suicide)||Ins...",31.7537,Shop Rite,-93.0836,1.0,NaN,1::21,0::Adult 18+||1::Adult 18+,0::Male||1::Male,NaN,"0::Killed||1::Unharmed, Arrested",0::Victim||1::Subject-Suspect
239674,2018-03-31,Louisiana,Gretna,1300 block of Cook Street,0,1,0::Unknown,0::Unknown,Shot - Wounded/Injured,29.9239,NaN,-90.0442,1.0,NaN,0::21,0::Adult 18+,0::Male,NaN,0::Injured,0::Victim
239675,2018-03-31,Texas,Houston,12630 Ashford Point Dr,1,0,0::Unknown,0::Unknown,"Shot - Dead (murder, accidental, suicide)",29.7201,Vanderbilt Court apartments,-95.6110,1.0,"Vic was found shot to death in car on 4/1/18, ...",0::42,0::Adult 18+,0::Male,NaN,0::Killed,0::Victim


In [6]:
data['total_victims'] = data['n_killed'] + data['n_injured']

In [7]:
gun_ = data.dropna(subset=['gun_type']).reset_index()[['gun_type', 'total_victims']]
gun_['gun_type'] = gun_['gun_type'].apply(lambda x: [g.replace('::', ':').split(':')[-1] for g in x.replace('||', '|').split('|')])
gun_list = pd.DataFrame.from_records(gun_.gun_type.tolist()).stack().reset_index()

gun_type_counter = Counter(gun_list[0])
gun_pd = pd.DataFrame.from_dict(gun_type_counter, orient='index').reset_index()
gun_pd = gun_pd.rename(columns= {'index': 'gun_type', 0: 'type_count'})


In [8]:
gun_pd.to_csv('gun_type_11102021.csv', index=False)

In [9]:
incident_character_list = "||".join(data['incident_characteristics'].dropna()).split("||")
incident_character_counter = Counter(incident_character_list)
ic_pd = pd.DataFrame.from_dict(incident_character_counter, orient='index').reset_index()
ic_pd = ic_pd.rename(columns= {'index': 'incidient_character', 0: 'incidient_count'})


In [10]:
ic_pd.to_csv('inc_chr_11102021.csv', index=False)

In [11]:
participant_relationship_list = data['participant_relationship'].str.replace('\d+', '')
participant_relationship_list = "::".join(participant_relationship_list.dropna()).split(":")
participant_relationship_list = "|".join(participant_relationship_list).split("|")
participant_relationship_counter = Counter(participant_relationship_list)
pr_pd = pd.DataFrame.from_dict(participant_relationship_counter, orient='index').reset_index()


<ipython-input-11-406852b5adb8>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  participant_relationship_list = data['participant_relationship'].str.replace('\d+', '')


In [12]:
pr_pd.to_csv('part_rel_11102021.csv', index=False)

In [13]:
data['participant_age'] = data['participant_age'].astype(str)

part_age = data['participant_age'].apply(lambda x: [a.replace('::', ':').split(':')[-1] for a in x.replace('||', '|').split('|')])
age_list = pd.DataFrame.from_records(part_age.tolist()).stack().reset_index()

pa_pd = pd.DataFrame(age_list[0])
pa_pd = pa_pd.rename(columns= { 0: 'age'})


In [14]:
pa_pd.to_csv('part_age_11102021.csv', index=False)

In [12]:
#Convert columns to string to apply regex filtering

data['participant_status'] = data['participant_status'].astype(str)
data['participant_age'] = data['participant_age'].astype(str)
data['participant_gender'] = data['participant_gender'].astype(str)
data['participant_age_group'] = data['participant_age_group'].astype(str)
data['participant_type'] = data['participant_type'].astype(str)
data['participant_relationship'] = data['participant_relationship'].astype(str)
data['gun_type'] = data['gun_type'].astype(str)
data['incident_characteristics'] = data['incident_characteristics'].astype(str)
data['participant_type'] = data['participant_type'].astype(str)
data['gun_stolen'] = data['gun_stolen'].astype(str)
data['location_description'] = data['location_description'].astype(str)

In [16]:
#Pull out values without the extra characters

#re.findall(r':([\d+]+)', data['participant_age'])
#re.findall(r':([A-Za-z0-9 \+-]+)', data['participant_age_group'])
#re.findall(r':([\w+]+)', data['participant_gender'])
#re.findall(r':([\w]+)' ,data['participant_status'][0])
#re.findall(r':([\w]+)' ,data['gun_type'][2])
#re.findall(r'[^||^]+' ,data['incident_characteristics'][8])
#re.findall(r'[A-Za-z]+', data['participant_type'][0])


In [13]:
def apply_regex(pattern_, column_):
    return re.findall(pattern_, column_)

In [14]:
data['participant_age'] = data['participant_age'].apply(lambda x: apply_regex(r':([\d+]+)', x))
data['participant_gender'] = data['participant_gender'].apply(lambda x: apply_regex(r':([A-Za-z]+)', x) )
data['participant_age_group'] = data['participant_age_group'].apply(lambda x: apply_regex(r':([A-Za-z0-9 \+-]+)', x))
data['participant_status'] = data['participant_status'].apply(lambda x: apply_regex(r'([A-Za-z]+)', x))
data['gun_type'] = data['gun_type'].apply(lambda x: apply_regex(r':([\w+\s*\.*\s*\d*\w*\[*\w*\-*\d*\]*]+)', x))
data['gun_stolen'] = data['gun_stolen'].apply(lambda x: apply_regex(r'([A-Za-z\s*\-*]+)', x))
data['incident_characteristics'] = data['incident_characteristics'].apply(lambda x : apply_regex(r'([^||^]+)', x))
data['participant_type'] = data['participant_type'].apply(lambda x : apply_regex(r'([A-Za-z\-*]+)', x))
data['participant_relationship'] = data['participant_relationship'].apply(lambda x: apply_regex(r'([A-Za-z\s*\-*]+)', x))

In [25]:
data['gun_type']

0                         []
1                         []
2         [Unknown, Unknown]
3                         []
4         [Handgun, Handgun]
                 ...        
239672             [Unknown]
239673             [Unknown]
239674             [Unknown]
239675             [Unknown]
239676    [Handgun, Shotgun]
Name: gun_type, Length: 239677, dtype: object

Count and separate the elements in the list into their own columns for easier use


In [15]:
#Group guns that are greater than 5 into 1 group

data['n_guns_involved'] = data['n_guns_involved'].apply(lambda x : "5+" if x > 5 else str(x))

In [16]:
#participant age group

data['adult'] = data['participant_age_group'].apply(lambda x: x.count('Adult 18+'))
data['teen'] = data['participant_age_group'].apply(lambda x: x.count('Teen 12-17'))
data['child'] = data['participant_age_group'].apply(lambda x: x.count('Child 0-11'))
data['group_total'] = data['child'] + data['teen'] + data['adult']

In [17]:
#gender

data['male'] = data['participant_gender'].apply(lambda x: x.count('Male'))
data['female'] = data['participant_gender'].apply(lambda x: x.count('Female'))

In [18]:
#gun stolen

data['stolen'] = data['gun_stolen'].apply(lambda x: x.count('Stolen'))
data['notstolen'] = data['gun_stolen'].apply(lambda x: x.count('Not-stolen'))
data['unknown'] = data['gun_stolen'].apply(lambda x: x.count('Unknown'))
data['NAN'] = data['gun_stolen'].apply(lambda x: x.count('nan'))

In [19]:
data['location_description'] = data['location_description'].str.translate(str.maketrans('','', string.punctuation))
data['location_description'] = data['location_description'].str.lower()

In [20]:
data.loc[data['location_description'].str.contains('walmart', case=False), 'location_description'] = 'walmart'
data.loc[data['location_description'].str.contains('mcdonald', case=False), 'location_description'] = 'mcdonalds'
data.loc[data['location_description'].str.contains('apartment', case=False), 'location_description'] = 'apartments'
data.loc[data['location_description'].str.contains('park', case=False), 'location_description'] = 'park'
data.loc[data['location_description'].str.contains('nightclub', case=False), 'location_description'] = 'nightclub'
data.loc[data['location_description'].str.contains('mall', case=False), 'location_description'] = 'mall'
data.loc[data['location_description'].str.contains('school', case=False), 'location_description'] = 'school'
data.loc[data['location_description'].str.contains('neighborhood', case=False), 'location_description'] = 'neighborhood'
data.loc[data['location_description'].str.contains('airport', case=False), 'location_description'] = 'airport'
data.loc[data['location_description'].str.contains('shop', case=False), 'location_description'] = 'shop'
data.loc[data['location_description'].str.contains('gas station', case=False), 'location_description'] = 'gas station'

In [21]:
#status

data['injured'] = data['participant_status'].apply(lambda x: x.count('Injured'))
data['arrested'] = data['participant_status'].apply(lambda x: x.count('Arrested'))
data['killed'] = data['participant_status'].apply(lambda x: x.count('Killed'))
data['unharmed'] = data['participant_status'].apply(lambda x: x.count('Unharmed'))

In [22]:
ch = data[data['city_or_county'] == 'Chicago']

In [43]:
il = data[data['state'] == 'Illinois']

In [45]:
sum(il['n_injured'] + il['n_killed'])

16923

In [40]:
sum(ch['n_killed'] + ch['n_injured'])

12531

In [27]:
#data.drop(columns=['gun_stolen','incident_characteristics', 'participant_relationship', 'participant_status', 'participant_type', 'participant_gender', 'participant_age_group'], inplace = True)

In [28]:
#from selenium import webdriver
#from webdriver_manager.chrome import ChromeDriverManager

#driver = webdriver.Chrome(ChromeDriverManager().install())

In [29]:
data.to_csv('gun_violence_cleaned_11102021.csv', index=False)

In [30]:
#gun type

#handgun
data['Handgun'] = data['gun_type'].apply(lambda x: x.count('Handgun'))
data['9mm'] = data['gun_type'].apply(lambda x: x.count('9mm'))
data['45_Auto'] = data['gun_type'].apply(lambda x: x.count('45 Auto'))
data['380_Auto'] = data['gun_type'].apply(lambda x: x.count('380 Auto'))
data['40_SW'] = data['gun_type'].apply(lambda x: x.count('40 SW'))
data['44_Magnum'] = data['gun_type'].apply(lambda x: x.count('44 Mag'))

#Rifle
data['Rifle'] = data['gun_type'].apply(lambda x: x.count('Rifle'))
data['22_LR'] = data['gun_type'].apply(lambda x: x.count('22 LR'))
data['AR-15'] = data['gun_type'].apply(lambda x: x.count('223 Rem [AR-15]'))
data['AK-47'] = data['gun_type'].apply(lambda x: x.count('7.62 [AK-47]'))
data['300_Winchester'] = data['gun_type'].apply(lambda x: x.count('300 Win'))

#Shotgun
data['Shotgun'] = data['gun_type'].apply(lambda x: x.count('Shotgun'))
data['12_gauge'] = data['gun_type'].apply(lambda x: x.count('12 gauge'))
data['20_gauge'] = data['gun_type'].apply(lambda x: x.count('20 gauge'))


data['Other'] = data['gun_type'].apply(lambda x: x.count('Other'))


In [31]:
#incidents

data['shot_injured'] = data['incident_characteristics'].apply(lambda x: x.count('Shot - Wounded/Injured'))
data['shot_dead'] = data['incident_characteristics'].apply(lambda x: x.count('Shot - Dead (murder, accidental, suicide)'))
data['no_shoot'] = data['incident_characteristics'].apply(lambda x: x.count('Non-Shooting Incident'))
data['shot_no_injured'] = data['incident_characteristics'].apply(lambda x: x.count('Shots Fired - No Injuries'))

data['possession'] = data['incident_characteristics'].apply(lambda x: x.count('Possession (gun(s) found during commission of other crimes)'))
data['armed_robbery'] = data['incident_characteristics'].apply(lambda x: x.count('Armed robbery with injury/death and/or evidence of DGU found'))

data['drug'] = data['incident_characteristics'].apply(lambda x: x.count('Drug involvement'))
data['driveby'] = data['incident_characteristics'].apply(lambda x: x.count('Drive-by (car to street, car to car)'))

data['domestic_violence'] = data['incident_characteristics'].apply(lambda x: x.count('Domestic Violence'))
data['home_invasion'] = data['incident_characteristics'].apply(lambda x: x.count('Home Invasion'))

data['mass_shoot'] = data['incident_characteristics'].apply(lambda x: x.count('Mass Shooting (4+ victims injured or killed excluding the subject/suspect/perpetrator, one location)'))

data['suicide'] = data['incident_characteristics'].apply(lambda x: x.count('Suicide'))

data['police'] = data['incident_characteristics'].apply(lambda x: x.count('Officer Involved Incident'))


In [32]:
#relationship

data['family'] = data['participant_relationship'].apply(lambda x: x.count('Family'))
data['driveby_random'] = data['participant_relationship'].apply(lambda x: x.count('Drive by - Random victims'))
data['aquaintance'] = data['participant_relationship'].apply(lambda x: x.count('Aquaintance'))
data['gang'] = data['participant_relationship'].apply(lambda x: x.count('Gang vs Gang'))
data['sigificant_other'] = data['participant_relationship'].apply(lambda x: x.count('Significant others - current or former'))
data['armed_robber'] = data['participant_relationship'].apply(lambda x: x.count('Armed Robbery'))
data['coworker'] = data['participant_relationship'].apply(lambda x: x.count('Co-worker'))
data['neighbor'] = data['participant_relationship'].apply(lambda x: x.count('Neighbor'))

data['home_invasion_random'] = data['participant_relationship'].apply(lambda x: x.count('Home Invasion - Perp Does Not Know Victim'))
data['home_invasion_known'] = data['participant_relationship'].apply(lambda x: x.count('Home Invasion - Perp Knows Victim'))

data['friends'] = data['participant_relationship'].apply(lambda x: x.count('Friends'))
data['mass_shooting'] = data['participant_relationship'].apply(lambda x: x.count('Mass shooting - Random victims'))